In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt

In [2]:
from rasterstats import zonal_stats


In [3]:
vector_path='../../sample_data/tr_boundry/AF_ADANA_2020_v10.gpkg'

In [4]:
adana=gpd.read_file('../../sample_data/tr_boundry/AF_ADANA_2020_v10.gpkg')
#adana=adana.iloc[0:200]

C:\Users\akif\.conda\envs\geo\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [13]:
dd=adana[0:5]
dd.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
raster_path='../../../Desktop/ubuntu_files/thinkswhere/geospatial/soil/TR_CurveNumberBasedRunoff_HYSOGs250m.tif'

In [7]:
%%time
input_raster = rio.open(raster_path)
# Read the raster values
array = input_raster.read(1)
# Get the affine
affine = input_raster.transform

Wall time: 60 ms


In [8]:
input_raster.crs

CRS.from_epsg(4326)

In [9]:
# Calculate zonal statistics
#https://automating-gis-processes.github.io/CSC/notebooks/L5/zonal-statistics.html
zz = zonal_stats(dd.geometry, array, affine=affine, stats=[ 'median'])

C:\Users\akif\.conda\envs\geo\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [10]:
zz[1]['median']

13.0

In [12]:
def calc_zonal(geom,array,affine):
    zonal=zonal_stats(geom, array, affine=affine, stats=[ 'median'])
    return zonal

In [68]:
%%time
adana['median_ndvi'] = adana.geometry.apply(lambda row:calc_zonal(row, array, affine=affine)[0]['median'])

C:\Users\akif\.conda\envs\geo\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Wall time: 2min 35s


In [10]:
adana.head()

,Il_Adi,Ilce_Adi,PlaceName,area,pixel_count,GrainPercentP3,GrainAreaP3,GrainClassP3,SilagePercentP3,SilageAreaP3,SilageClassP3,geometry
0,ADANA,YÜREĞİR,Agzibuyuk,2.668082,266.0,0.011278,0.030091,False,0.0,0.0,False,"POLYGON ((35.44724 36.86743, 35.44610 36.86737..."
1,ADANA,YÜREĞİR,Agzibuyuk,2.936688,294.0,0.000000,0.000000,False,0.0,0.0,False,"POLYGON ((35.44606 36.86887, 35.44598 36.86886..."
2,ADANA,YÜREĞİR,Agzibuyuk,1.527408,148.0,0.033784,0.051602,False,0.0,0.0,False,"POLYGON ((35.44644 36.86474, 35.44685 36.86509..."
3,ADANA,YÜREĞİR,Agzibuyuk,2.117166,211.0,0.004739,0.010034,False,0.0,0.0,False,"POLYGON ((35.44599 36.86972, 35.44596 36.86972..."
4,ADANA,YÜREĞİR,Agzibuyuk,1.218371,123.0,0.000000,0.000000,False,0.0,0.0,False,"POLYGON ((35.44698 36.87101, 35.44693 36.87105..."


In [11]:
cp_adana=adana.copy()

In [12]:
import multiprocessing as mp
from multiprocessing import Pool

# Function that parallelizes the apply function among the cores of the CPU
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Function that adds a sea_name column in the main dataframe
def add_features(df):
    # Apply the function
    #df['sea_name'] = df.apply(lambda x: get_seaname(x['Position Longitude'], x['Position Latitude']), axis=1)
    df['median_ndvi'] = df.geometry.apply(lambda row:calc_zonal(row, array, affine=affine)[0]['median'])
    return df

In [ ]:
%%time
### Apply the above function to the dataframe
ships_df = parallelize_dataframe(cp_adana, add_features, n_cores=5)

In [73]:
with mp.Pool() as pool:
    results = pool.map(calc_dist2, grp_lst_args)

In [16]:
import multiprocessing as mp
from multiprocessing import Pool

# Function that parallelizes the apply function among the cores of the CPU
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Function that adds a sea_name column in the main dataframe
def add_features(df):
    # Apply the function
    #df['sea_name'] = df.apply(lambda x: get_seaname(x['Position Longitude'], x['Position Latitude']), axis=1)
    df['ppp'] = df.pixel_count.apply(lambda row: row*2)
    print('hey')
    return df

In [ ]:
parallelize_dataframe(dd, add_features, n_cores=5)

In [2]:
from multiprocessing import Pool

def f(x):
    return x*x


In [4]:
def run():
    pool = Pool(processes=4)              
    print(pool.map(f, range(10))  )
    return 0

In [ ]:
run()